In [2]:
import warp as wp
from warp.sparse import bsr_zeros

wp.init()

# ------------------------
# 系统维度
# ------------------------
N = 1000  # 大规模矩阵，N 个自由度块
# 对应总自由度 = N * block_size

# ------------------------
# 1. 3x3 块的 BSR 矩阵
# ------------------------
block_size_3x3 = 3
A_3x3 = bsr_zeros(
    rows_of_blocks=N,
    cols_of_blocks=N,
    block_type=wp.mat33  # 3x3 块
)
print("\n3x3 BSR matrix:")
print("Shape of blocks:", A_3x3.block_shape)
print("Matrix shape:", A_3x3.shape)


print('\n')


from warp.sparse import bsr_from_triplets

# --------------------------
# 定义 COO triplets
# --------------------------
# 假设我们要构建一个 4x4 的矩阵，由 2x2 块组成
# 总共有 2x2 = 4 个块
rows = wp.array([0, 0, 1, 1], dtype=int)  # 块的行索引
cols = wp.array([0, 1, 0, 1], dtype=int)  # 块的列索引

# 每个块是 2x2 矩阵
vals = wp.array([
    [[1.0, 0.5],
     [0.5, 2.0]],   # 块 (0,0)
    [[0.0, 1.0],
     [1.0, 0.0]],   # 块 (0,1)
    [[2.0, 0.0],
     [0.0, 2.0]],   # 块 (1,0)
    [[1.0, -1.0],
     [-1.0, 1.0]]   # 块 (1,1)
], dtype=wp.float32)

# --------------------------
# 创建 BSR 矩阵
# --------------------------
A = bsr_from_triplets(
    rows_of_blocks=2,      # 行块数
    cols_of_blocks=2,      # 列块数
    rows=rows,             # 块行索引
    columns=cols,          # 块列索引
    values=vals            # 块数据
)

# --------------------------
# 查看矩阵信息
# --------------------------
print("BSR matrix shape (blocks):", A.nrow, "x", A.ncol)
print("Block shape:", A.block_shape)
print("Values array:\n", A.values)

# 可以获取压缩行索引和列索引
nnz = A.nnz_sync()
print("Exact number of non-zero blocks:", nnz)
print("Row indices of blocks:", A.uncompress_rows()[:nnz])
print("Column indices of blocks:", A.columns[:nnz])
print(A)

print('\n')

# --------------------------
# 定义 COO triplet，只一个非零块
# --------------------------
rows = wp.array([0], dtype=int)  # 块的行索引
cols = wp.array([0], dtype=int)  # 块的列索引

# 块大小 2x2
vals = wp.array([
    [[1.0, 0.0],
     [0.0, 1.0]]   # 块 (0,0)
], dtype=wp.float32)

# --------------------------
# 创建 BSR 矩阵
# --------------------------
A = bsr_from_triplets(
    rows_of_blocks=2,  # 2x2 块矩阵 → 总矩阵 4x4
    cols_of_blocks=2,
    rows=rows,
    columns=cols,
    values=vals
)

# --------------------------
# 查看矩阵信息
# --------------------------
nnz = A.nnz_sync()
print("Exact number of non-zero blocks:", nnz)
print("Row indices of blocks:", A.uncompress_rows()[:nnz])
print("Column indices of blocks:", A.columns[:nnz])
print("Values array:\n", A.values)
print(A)



print('\n')

# --------------------------
# COO triplets，包含重复元素
# --------------------------
rows = wp.array([0, 0, 1], dtype=int)   # 行索引
cols = wp.array([1, 1, 2], dtype=int)   # 列索引
vals = wp.array([2.0, 3.0, 4.0], dtype=float)  # 对应值

# --------------------------
# 创建 BSR 矩阵，1x1 块（CSR）
# --------------------------
A = bsr_from_triplets(
    rows_of_blocks=2,  # 行块数 → 总矩阵 2x2 块，每块 1x1 → 总矩阵 2x2
    cols_of_blocks=3,  # 列块数 → 总矩阵 2x3
    rows=rows,
    columns=cols,
    values=vals
)

print(A)


3x3 BSR matrix:
Shape of blocks: (3, 3)
Matrix shape: (3000, 3000)


BSR matrix shape (blocks): 2 x 2
Block shape: (2, 2)
Values array:
 [[[ 1.   0.5]
  [ 0.5  2. ]]

 [[ 0.   1. ]
  [ 1.   0. ]]

 [[ 2.   0. ]
  [ 0.   2. ]]

 [[ 1.  -1. ]
  [-1.   1. ]]]
Exact number of non-zero blocks: 4
Row indices of blocks: [0 0 1 1]
Column indices of blocks: [0 1 0 1]
BsrMatrix_float32_2_2(
	nrow=2,
	ncol=2,
	nnz=4,
	offsets=[0 2 4],
	columns=[0 1 0 1],
	values=[[[ 1.   0.5]
  [ 0.5  2. ]]

 [[ 0.   1. ]
  [ 1.   0. ]]

 [[ 2.   0. ]
  [ 0.   2. ]]

 [[ 1.  -1. ]
  [-1.   1. ]]],
)


Exact number of non-zero blocks: 1
Row indices of blocks: [0]
Column indices of blocks: [0]
Values array:
 [[[1. 0.]
  [0. 1.]]]
BsrMatrix_float32_2_2(
	nrow=2,
	ncol=2,
	nnz=1,
	offsets=[0 1 1],
	columns=[0],
	values=[[[1. 0.]
  [0. 1.]]],
)


BsrMatrix_float32(
	nrow=2,
	ncol=3,
	nnz=3,
	offsets=[0 1 2],
	columns=[1 2 4],
	values=[5. 4. 0.],
)


In [3]:
import warp as wp
from warp.sparse import bsr_from_triplets
from warp.optim.linear import cg


# 初始化 Warp
wp.init()

# --------------------------
# 定义 BSR 稀疏矩阵 (4x4, 2x2 块)
# --------------------------
rows = wp.array([0, 0, 1, 1], dtype=int)  # 块行索引
cols = wp.array([0, 1, 0, 1], dtype=int)  # 块列索引

vals = wp.array([
    [[4.0, 1.0],
     [1.0, 3.0]],   # 块 (0,0)
    [[1.0, 0.0],
     [0.0, 1.0]],   # 块 (0,1)
    [[1.0, 0.0],
     [0.0, 2.0]],   # 块 (1,0)
    [[3.0, 1.0],
     [1.0, 2.0]]    # 块 (1,1)
], dtype=wp.float32)

# 创建 BSR 矩阵
A = bsr_from_triplets(
    rows_of_blocks=2,   # 行块数
    cols_of_blocks=2,   # 列块数
    rows=rows,
    columns=cols,
    values=vals
)

# --------------------------
# 构造右端向量 b 和初值 x
# --------------------------
# 总维度 = 行块数 * 每块行数 = 2 * 2 = 4
b = wp.array([1.0, 2.0, 3.0, 4.0], dtype=wp.float32)
x = wp.zeros_like(b)

# --------------------------
# 调用共轭梯度求解器
# --------------------------
# tol: 误差容忍度
# max_iters: 最大迭代次数
result = cg(A, b, x, tol=1e-6, maxiter=100)

# --------------------------
# 打印结果
# --------------------------
print("Solution x =", x.numpy())
print(result)


Solution x = [ 1.6666901e-01 -2.4097676e-06  3.3333063e-01  1.8333383e+00]
(20, 4.872774408170143e-06, 5.477225801645495e-06)


In [30]:
from warp.sparse import bsr_zeros, bsr_identity, bsr_diag

wp.mat33(0.0)
wp.vec3(0.0,0.0,1.0)
I = bsr_identity(rows_of_blocks=4, block_type=wp.float32)
print(I)

BsrMatrix_float32(
	nrow=4,
	ncol=4,
	nnz=4,
	offsets=[0 1 2 3 4],
	columns=[0 1 2 3],
	values=[1. 1. 1. 1.],
)


In [ ]:
import warp as wp
from warp.sparse import bsr_from_triplets

wp.init()

# -----------------------------
# 1. 构建一个 4x4 BSR 矩阵，块大小 2x2
# -----------------------------
# Triplets: 行索引, 列索引, 2x2 块数据
rows = wp.array([0, 1], dtype=int)
cols = wp.array([0, 1], dtype=int)
blocks = wp.array([[[1.0,2.0],[7.0,8.0]], [[3.0,4.0],[5.0,6.0]]], dtype=wp.mat22)

A = bsr_from_triplets(
    rows_of_blocks=2,  # 2x2 块矩阵 → 总矩阵 4x4
    cols_of_blocks=2,
    rows=rows,
    columns=cols,
    values=blocks
)

print("原始 BSR 数据:")
print(A.values)  # A.values 存储所有非零块

# -----------------------------
# 2. GPU kernel: 原地修改第一个块的 (0,1) 元素
# -----------------------------
@wp.kernel
def modify_bsr_element(A: wp.array(dtype=float)):
    # 假设只修改第一个块的元素
    A[0][1] = wp.mat22([[99.0, 0.0], [0.0, 99.0]])

# launch kernel
wp.launch(modify_bsr_element, dim=1, inputs=[A.values])

print("修改后的 BSR 数据:")
print(A.values)


原始 BSR 数据:
[[[1. 2.]
  [7. 8.]]

 [[3. 4.]
  [5. 6.]]]
Module __main__ 51852c2 load on device 'cpu' took 0.37 ms  (error)


WarpCodegenError: Error while parsing function "modify_bsr_element" at C:\Users\baiix\AppData\Local\Temp\ipykernel_16104\275007628.py:31:
    A_values[0] = wp.mat22([[99.0, 0.0], [0.0, 99.0]])
;Construct `ast.List` not supported in kernels.